In [ ]:
import dataframeLoader as dfl
import datetime as dt
from dateutil.parser import parse
from pathlib import Path

import plotly
import pandas as pd

from importlib import reload 
reload(dfl)
# generate daily reports for all appliances between the below dates
root = '.dataDir'
fromdt = parse('2024-10-07')
todt = parse('2024-10-07')
range = pd.date_range(fromdt,todt,freq='d')

# metricsArr = ['cpu_used', 'download_workers_count', 'memory_used', 'task_queue_length', 'infra_access_latency', 'pod_cpu_usage', 'pod_memory_usage'] 
metricsArr = ['cpu_used','task_queue_length', 'memory_used']
metrics_category_order = [
            "dataScannedinGB",
            "numberOfColsScanned", "numberOfChunksScanned", 
             "scanTime", "fileDownloadTimeInHrs","uniqPodCount",
             "numFilesScanned", "avgFileSizeInMB", "IdleTimeInHrs",
             "cpu_used_avg", "memory_used_avg",
             "tmp_taskq_avg", "taskq_max"
             ]

for rptDt in range:
    datestr = rptDt.date().__str__()
    daterange=[datestr, datestr]
    df = dfl.loadApplianceTimeSeriesData(root, metricsArr, daterange)
    for appliance in list(df.appliance_id.unique()):
        title = 'Appliance plot for appliance_id '+appliance+' on '+datestr
        dfp = df[(df['appliance_id'] == appliance)]
        if len(set(dfp.metrics.unique()).intersection(metrics_category_order)) > 5:
            dirPath='appliancePlots/'+appliance
            Path(dirPath).mkdir(parents=True, exist_ok=True)
            fig  = dfl.plotMetricsFacetForApplianceId(dfp, title, metrics_category_order)
            plotly.offline.plot(fig, filename=dirPath+'/'+datestr+'.html', auto_open=False)
